###  CS439 Final Project - TDA + ML using USDA Food Environment Atlas (2019) + Census Tract list (2025)
Denise Gonzalez-Cruz,   netid: dg1070,    section: 08

notes:
- This notebook targets: both Census Tract list (2025) + USDA Food Environment Atlas (2019), with possible ACS (American Community Survey) integration
- Mapping support: both TIGER/Line shapefile (.shp) AND GeoJSON are supported (we will try GeoJSON first if available).
- TDA pipeline: GUDHI (unweighted + weighted Rips)

Final notes:
- This notebook focuses on GUDHI-only, weighted-first VR complexes using weighting focused on socioeconomic features
- The slowest step is computing local per-tract diagrams. We can consider subsampling or increasing k for speed.
- can easily change alpha_poverty/alpha_income/alpha_density at the top to experiment with different outcomes

Overall, this code had to be redone as the original data I was using was the ACS ZTCA which groups areas together instead of keeping the data at the census tract level. I was able to find and properly merge the data in the NJ and other jupyter notebooks

In [2]:
# 1) imports
import os, time, warnings
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# TDA: GUDHI
USE_GUDHI = True
try:
    import gudhi as gd
    import gudhi.representations as gdr
except Exception as e:
    USE_GUDHI = False
    warnings.warn(f'GUDHI import failed: {e}. Install GUDHI to run the TDA sections.')

import matplotlib.pyplot as plt

In [17]:
# grouping the state and county data by zipcode 
atlas = pd.read_csv('2025-food-environment-atlas-data/StateAndCountyData.csv')
#print(atlas_data)
df_wide = atlas.pivot_table(
        index="FIPS",
        columns="Variable_Code",
        values="Value",
        aggfunc="first"
).reset_index()

# Keep FIPS–State–County metadata 
meta = atlas[["FIPS", "State", "County"]].drop_duplicates()

# Merge to restore those columns
df_wide = meta.merge(df_wide, on="FIPS", how="left")

# Reorder so these are the first 3 columns
first = ["FIPS", "State", "County"]
other = [c for c in df_wide.columns if c not in first]

df_wide = df_wide[first + other]

# changin type of FIPS to str and padding with leading zeros
df_wide['FIPS'] = df_wide['FIPS'].apply(lambda x: str(x).zfill(5) if pd.notnull(x) else x)

# dropping alaska areas since they don't map to traditional fips codes, and are relatively few in number when it comes to US as a whole
# this may affect analysis if we were to focus on alaska, but for now we drop them
df_atlas= df_wide.dropna(subset=['FIPS']) # dropping n/a fips entries 
df_atlas

,FIPS,State,County,AGRITRSM_OPS12,AGRITRSM_OPS17,AGRITRSM_RCT12,AGRITRSM_RCT17,BERRY_ACRES12,BERRY_ACRES17,BERRY_ACRESPTH12,...,VEG_ACRESPTH12,VEG_ACRESPTH17,VEG_FARMS12,VEG_FARMS17,VLFOODSEC_18_20,VLFOODSEC_21_23,WICS16,WICS22,WICSPTH16,WICSPTH22
0,1001.0,AL,Autauga,10.0,6.0,146000.0,66000.0,5.0,4.0,0.090269,...,22.206174,21.808991,45.0,31.0,5.2,4.4,5.0,4.0,0.090828,0.068072
1,1003.0,AL,Baldwin,16.0,19.0,204000.0,337000.0,93.0,113.0,0.437604,...,9.213207,7.773349,50.0,39.0,5.2,4.4,29.0,27.0,0.145356,0.115671
2,1005.0,AL,Barbour,32.0,11.0,304000.0,71000.0,42.0,6.0,1.669515,...,1.629765,2.424772,7.0,18.0,5.2,4.4,9.0,9.0,0.338168,0.361780
3,1007.0,AL,Bibb,6.0,3.0,21000.0,-9999.0,-9999.0,21.0,-9999.000000,...,0.620843,0.221729,11.0,5.0,5.2,4.4,5.0,4.0,0.221513,0.179767
4,1009.0,AL,Blount,8.0,13.0,30000.0,50000.0,38.0,36.0,0.657587,...,11.715438,10.002250,64.0,54.0,5.2,4.4,8.0,5.0,0.138639,0.084635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3151,9150.0,CT,Northeastern Connecticut,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.000000,...,-8888.000000,-8888.000000,-8888.0,-8888.0,-8888.0,4.4,NaN,NaN,NaN,NaN
3152,9160.0,CT,Northwest Hills,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.000000,...,-8888.000000,-8888.000000,-8888.0,-8888.0,-8888.0,4.4,NaN,NaN,NaN,NaN
3153,9170.0,CT,South Central Connecticut,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.000000,...,-8888.000000,-8888.000000,-8888.0,-8888.0,-8888.0,4.4,NaN,NaN,NaN,NaN
3154,9180.0,CT,Southeastern Connecticut,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.0,-8888.000000,...,-8888.000000,-8888.000000,-8888.0,-8888.0,-8888.0,4.4,NaN,NaN,NaN,NaN


In [ ]:
# excel census data to csv - with row 1 as the header
# using 2024-2025 tracts 
census_data = pd.read_excel('CensusTractList2025.xlsx', sheet_name='2024-2025 tracts')

census_data.to_csv('census_tracts_2025.csv', index=False)
census_df = pd.read_csv('census_tracts_2025.csv', dtype={'GEOID': str})

census_df

,Year,MSA/MD code type,MSA/MD code,State code,County code,Tract,MSA/MD name,State,County name,FIPS code,MSA/MD MFI,Tract MFI,Tract income percentage,Tract income level
0,2024-2025,MSA,10180,48,59,30101,"ABILENE, TX",TX,CALLAHAN COUNTY ...,48059030101,68388,61923.0,90.54,Middle
1,2024-2025,MSA,10180,48,59,30102,"ABILENE, TX",TX,CALLAHAN COUNTY ...,48059030102,68388,66132.0,96.70,Middle
2,2024-2025,MSA,10180,48,59,30200,"ABILENE, TX",TX,CALLAHAN COUNTY ...,48059030200,68388,59531.0,87.04,Middle
3,2024-2025,MSA,10180,48,253,20101,"ABILENE, TX",TX,JONES COUNTY ...,48253020101,68388,55179.0,80.68,Middle
4,2024-2025,MSA,10180,48,253,20102,"ABILENE, TX",TX,JONES COUNTY ...,48253020102,68388,0.0,0.00,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85524,2024-2025,Non-MSA,99999,78,30,960900,NaN,VI,SAINT THOMAS ISLAND ...,78030960900,52000,51797.0,99.60,Middle
85525,2024-2025,Non-MSA,99999,78,30,961000,NaN,VI,SAINT THOMAS ISLAND ...,78030961000,52000,38125.0,73.31,Moderate
85526,2024-2025,Non-MSA,99999,78,30,961100,NaN,VI,SAINT THOMAS ISLAND ...,78030961100,52000,38158.0,73.38,Moderate
85527,2024-2025,Non-MSA,99999,78,30,961200,NaN,VI,SAINT THOMAS ISLAND ...,78030961200,52000,36188.0,69.59,Moderate


Parameters: these include weights for the distance metric that will account for socioeconomic factors

In [ ]:
# 2) parameters + file paths
STATE_POSTAL = 'NJ'
TRACT_LIST_CSV = 'census_tracts_2025.csv'   # tract list (contains GEOID or state/county/tract cols - tracts for 2025: State code,County code,Tract)
FEA_2019_CSV = 'Food_Environment_Atlas_2019.csv'  # USDA FEA (county-level)
ACS_CSV = 'ACSST5Y2023.S1901-Data.csv'                   # ACS tract-level CSV with B01003 (population) and GEOID
TIGER_SHP = 'tl_2022_34_tract.shp'               # optional TIGER shapefile for NJ (FIPS=34)
GEOJSON_PATH = 'nj_tracts.geojson'               # optional geojson
OUTPUT_DIR = 'cs439_finalProject/project_outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# weights for distance metric - used in 10)
alpha_poverty = 0.2
alpha_income = 0.1
alpha_density = 0.1

# GUDHI parameters
MAX_DIM = 1   # compute H0 and H1 homology classes
MAX_EDGE = None  # will be set from weighted distance matrix

# neighborhood size for local features
K_NEIGHBORS = 12

In [ ]:
# 3) load data files
print('Loading tract list...')
tracts = pd.read_csv(TRACT_LIST_CSV, dtype=str)
tracts['County code'].strip()
print('Tracts shape:', tracts.shape)

print('Loading Food Environment Atlas (2019)...')
fea = pd.read_csv(FEA_2019_CSV, dtype=str)
print('FEA shape:', fea.shape)

print('Loading ACS 5-year (for population/density)...')
acs = pd.read_csv(ACS_CSV, dtype=str)
print('ACS shape:', acs.shape)

In [ ]:
# 4) normalize/construct GEOID in tract list and ACS
print(' Constructing/ensuring GEOID fields...')

def zero_pad(x,w):
    return str(x).zfill(w)

# ensure GEOID in tracts
if 'GEOID' not in tracts.columns:
    # heuristics
    state_col = 'State code'
    county_col = 'County code'
    tract_col = 'Tract'
    if state_col and county_col and tract_col:
        tracts['STATEFP'] = tracts[state_col].apply(lambda x: zero_pad(x,2))
        tracts['COUNTYFP'] = tracts[county_col].apply(lambda x: zero_pad(x,3))
        tracts['TRACTCE'] = tracts[tract_col].apply(lambda x: zero_pad(x,6))
        tracts['GEOID'] = tracts['STATEFP'] + tracts['COUNTYFP'] + tracts['TRACTCE']
    else:
        # try to find any 11-digit column
        for c in tracts.columns:
            vals = tracts[c].dropna().astype(str)
            if vals.str.len().isin([11]).all():
                tracts['GEOID'] = vals
                break

tracts['GEOID'] = tracts['GEOID'].astype(str)
tracts['COUNTYFIPS'] = tracts['GEOID'].str[:5]
print('Unique state prefixes:', tracts['GEOID'].str[:2].unique())

# ensure GEOID in ACS
acs_cols = [c.lower() for c in acs.columns]
geoid_col = next((c for c in acs.columns if c.lower()=='geoid' or c.lower().endswith('geoid')), None)
if geoid_col is None:
    raise ValueError('ACS CSV must include a GEOID column for tracts (11-digit).')
acs[geoid_col] = acs[geoid_col].astype(str).str.zfill(11)
acs.rename(columns={geoid_col:'GEOID'}, inplace=True)

In [ ]:
# 5) filter to the selected state (NJ) 
## chose NJ based on our current location and data availabiiity - may be a good start for more urban areas/mixed settings
print('Filtering to', STATE_POSTAL)
postal_to_fips = {'NJ':'34'}
state_fips = postal_to_fips[STATE_POSTAL]
tracts = tracts[tracts['GEOID'].str.startswith(state_fips)].copy()
acs = acs[acs['GEOID'].str.startswith(state_fips)].copy()
print('Filtered tracts:', tracts.shape, 'Filtered ACS:', acs.shape)

In [ ]:
# building off of ph_computations_unweighted from  https://github.com/deniseg20/MSRIUP_TDA_Healthcare_Accessibility - ph_computations_unweighted.ipynb cell 10
import geopandas as gpd

# Census tract shapefile (national)
shpfilename = "https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_us_tract.zip"
tracts = gpd.read_file(shpfilename)

# filter for NJ
nj = tracts[tracts["STATEFP"] == "34"]

# reproject
nj = nj.to_crs("EPSG:4326")

# load your processed ACS dataset
gdf_vars = gpd.GeoDataFrame(acs, geometry=None) 

# Merge shapefile + attributes
nj_merged = nj.merge(gdf_vars, left_on="GEOID", right_on="FIPS", how="inner")


In [ ]:
# 6) merging datasets: tracts + ACS + FEA (via county FIPS)
print('Merging datasets...')
# FEA county fips detection
fea_cols_lower = [c.lower() for c in fea.columns]
county_fips_col = None
for cand in ('countyfips','county_fips','fips','stcofips'):
    if cand in fea_cols_lower:
        county_fips_col = fea.columns[fea_cols_lower.index(cand)]
        break
if county_fips_col is None:
    raise ValueError('Could not find county FIPS in FEA CSV.')

fea[county_fips_col] = fea[county_fips_col].astype(str).str.zfill(5)

# merging tracts with ACS by GEOID
merged = tracts.merge(acs, on='GEOID', how='left', suffixes=('','_acs'))

# attaching FEA by county
merged['COUNTYFIPS'] = merged['GEOID'].str[:5]
merged = merged.merge(fea, left_on='COUNTYFIPS', right_on=county_fips_col, how='left', suffixes=('','_fea'))

print('Merged shape:', merged.shape)
merged.to_csv(os.path.join(OUTPUT_DIR,'merged_initial.csv'), index=False)
print('Saved merged_initial.csv')

In [ ]:
# 7) compute population density from ACS + tract shapefile area (ALAND) from ACS or shapefile
# chosen approach: using ACS population (B01003 or similar) and 'ALAND' from shapefile. If ALAND is not in ACS, read the shapefile.
print('Computing population density...')
# find population column in ACS
pop_col = next((c for c in acs.columns if c.lower().startswith('b01003') or 'population' in c.lower() or 'total'==c.lower()), None)
if pop_col is None:
    pop_col = next((c for c in acs.columns if 'population' in c.lower()), None)

if pop_col is None:
    raise ValueError('Could not detect a population column in ACS CSV (e.g., B01003).')

merged['population'] = pd.to_numeric(merged[pop_col].fillna(0), errors='coerce')

# get land area (from shapefile if ACS lacks ALAND)
if 'ALAND' in merged.columns:
    merged['ALAND'] = pd.to_numeric(merged['ALAND'], errors='coerce')
else:
    if os.path.exists(GEOJSON_PATH):
        gdf = gpd.read_file(GEOJSON_PATH)
    elif os.path.exists(TIGER_SHP):
        gdf = gpd.read_file(TIGER_SHP)
    else:
        gdf = None

    if gdf is not None:
        if 'GEOID' not in gdf.columns:
            gdf['GEOID'] = gdf['STATEFP'] + gdf['COUNTYFP'] + gdf['TRACTCE']
        gdf = gdf[['GEOID','geometry']].copy()
        # ensure projections for area calculation
        gdf = gdf.to_crs(epsg=3857)
        gdf['ALAND'] = gdf['geometry'].area  # area in m^2
        # merge ALAND back to merged
        merged = merged.merge(gdf[['GEOID','ALAND']], on='GEOID', how='left')
    else:
        raise ValueError('No shapefile or geojson available to compute land area. Provide TIGER shapefile or GeoJSON.')

# convert ALAND (m^2) to km^2
merged['ALAND_km2'] = pd.to_numeric(merged['ALAND'], errors='coerce') / 1e6
merged['density'] = merged['population'] / merged['ALAND_km2']
merged['density'] = merged['density'].replace([np.inf, -np.inf], np.nan).fillna(0)

print('Density computed. Summary:')
print(merged['density'].describe())

In [ ]:
# 8) select poverty and income fields
# heuristics: find columns with 'poverty' and 'income'
poverty_col = next((c for c in merged.columns if 'poverty' in c.lower()), None)
income_col = next((c for c in merged.columns if 'income' in c.lower() and 'median' in c.lower()), None)

if poverty_col is None:
    # ACS standard fields (e.g., B17001 or S1701) 
    poverty_col = next((c for c in merged.columns if 'b17001' in c.lower() or 'poverty' in c.lower()), None)

if income_col is None:
    income_col = next((c for c in merged.columns if 'median' in c.lower() and 'income' in c.lower()), None)

print('Detected poverty col:', poverty_col, 'Detected income col:', income_col)

# coerce to numeric and fillna
merged['poverty_rate'] = pd.to_numeric(merged[poverty_col], errors='coerce') if poverty_col else 0
merged['median_income'] = pd.to_numeric(merged[income_col], errors='coerce') if income_col else np.nan
merged['poverty_rate'] = merged['poverty_rate'].fillna(merged['poverty_rate'].median())
merged['median_income'] = merged['median_income'].fillna(merged['median_income'].median())

# standardize socioeconomic variables before weighting to prevent imbalance
scaler = StandardScaler()
scaler_cols = ['poverty_rate','median_income','density']
merged[['poverty_s','income_s','density_s']] = scaler.fit_transform(merged[scaler_cols].fillna(0))

In [ ]:
# 9) compute geographic centroids and pairwise geographic distances (km)
print('Computing tract centroids and geographic pairwise distances...')
if 'geometry' not in locals():
    # load shapefile/geojson again to obtain geometry if not present
    if os.path.exists(GEOJSON_PATH):
        gdf = gpd.read_file(GEOJSON_PATH)
    else:
        gdf = gpd.read_file(TIGER_SHP)

    if 'GEOID' not in gdf.columns:
        gdf['GEOID'] = gdf['STATEFP'] + gdf['COUNTYFP'] + gdf['TRACTCE']

    gdf = gdf[['GEOID','geometry']]

# ensure same CRS for centroid calculation
gdf = gdf.to_crs(epsg=3857)
gdf['centroid'] = gdf.geometry.centroid

# merge centroids onto merged table
gdf_cent = gdf[['GEOID','centroid']].copy()
merged = merged.merge(gdf_cent, on='GEOID', how='left')
merged['centroid'] = merged['centroid'].apply(lambda x: x if not pd.isna(x) else None)

# build numpy array of lon/lat in EPSG:4326 for distances
# convert centroids to lat/lon
gdf_latlon = gdf.copy().to_crs(epsg=4326)
coords = np.array([[pt.y, pt.x] for pt in gdf_latlon['centroid']])  # lat, lon

# map GEOIDs to row order in merged
g_order = merged['GEOID'].tolist()
# create mapping from GEOID to centroid (lat,lon)
geo_to_latlon = {g: (np.nan, np.nan) for g in g_order}
for idx, row in gdf_latlon.iterrows():
    geo_to_latlon[row['GEOID']] = (row['centroid'].y, row['centroid'].x)

coords_ordered = np.array([geo_to_latlon[g] for g in g_order])

# compute haversine distances (km)
def haversine_array(latlon):
    # latlon: (n,2) with (lat, lon) in degrees
    lat = np.radians(latlon[:,0])
    lon = np.radians(latlon[:,1])
    dlat = lat[:,None] - lat[None,:]
    dlon = lon[:,None] - lon[None,:]
    a = np.sin(dlat/2)**2 + np.cos(lat)[:,None]*np.cos(lat)[None,:]*np.sin(dlon/2)**2
    R = 6371.0
    d = 2*R*np.arcsin(np.sqrt(a))
    return d

D_geo = haversine_array(coords_ordered)  # (n,n) in km
print('Geographic distance matrix computed; shape:', D_geo.shape)

In [ ]:
# 10) build weighted distance matrix
print('Building weighted distance matrix (Option 2)...')

poverty_s = merged['poverty_s'].to_numpy(dtype=float)
income_s = merged['income_s'].to_numpy(dtype=float)
density_s = merged['density_s'].to_numpy(dtype=float)

# pairwise absolute differences
from scipy.spatial.distance import pdist, squareform

D_pov = squareform(pdist(poverty_s.reshape(-1,1), metric='cityblock'))
D_inc = squareform(pdist(income_s.reshape(-1,1), metric='cityblock'))
D_den = squareform(pdist(density_s.reshape(-1,1), metric='cityblock'))

# distance metric - 
D_weighted = D_geo + alpha_poverty * D_pov + alpha_income * D_inc + alpha_density * D_den

# Normalize or scale if needed (optional). We'll set MAX_EDGE to 90th percentile to limit complex size
MAX_EDGE = np.percentile(D_weighted[np.triu_indices_from(D_weighted, k=1)], 90)
print('Weighted distance matrix built. MAX_EDGE set to 90th percentile:', MAX_EDGE)

# Save small sample for reproducibility
np.save(os.path.join(OUTPUT_DIR,'D_weighted.npy'), D_weighted)

In [ ]:

# 11) build VR complex from distance matrix and compute persistence
if not USE_GUDHI:
    raise RuntimeError('GUDHI is not available. Install GUDHI to continue.')

print('Creating GUDHI RipsComplex from distance matrix...')
start = time.time()
rips = gd.RipsComplex(distance_matrix=D_weighted, max_edge_length=MAX_EDGE)
simplicial_tree = rips.create_simplex_tree(max_dimension=MAX_DIM)
print('Number of simplices:', simplicial_tree.num_simplices())
print('Computing persistence...')
pers = simplicial_tree.persistence()
print('Persistence computed in %.2f s' % (time.time()-start))

# plot persistence diagram
gd.plot_persistence_diagram(pers)
plt.title('GUDHI: Weighted VR persistence (H0,H1)')
plt.show()



In [ ]:
# 12) extract persistence features and death simplices
print('Extracting persistence features (global) and death simplices...')
# convert to diagram array for H0/H1 separation - to view in different homology classes
diag = simplicial_tree.persistence_intervals_in_dimension

# helper to extract diagrams per dimension
def get_diagram(stree, dim):
    return np.array([pt[1] for pt in stree.persistence() if stree.persistence()[0] is not None])

# simplicial_tree.persistence() returns persistence as list of (dim, (birth, death)) 
# iterate for H0 (0-dim homology class) and H1 (1-dim homology class)
h0 = []
h1 = []
for d in pers:
    dim = d[0]
    b, de = d[1]
    if de == float('inf'):
        de = MAX_EDGE*1.1
    if dim == 0:
        h0.append((b,de))
    elif dim == 1:
        h1.append((b,de))

h0 = np.array(h0) if len(h0)>0 else np.empty((0,2))
h1 = np.array(h1) if len(h1)>0 else np.empty((0,2))

# persistence stats (global)
def pers_stats(dgm):
    if dgm.size==0:
        return {'n':0,'mean':0,'max':0,'tot':0}
    pers = dgm[:,1] - dgm[:,0]
    return {'n':len(pers), 'mean':np.mean(pers), 'max':np.max(pers), 'tot':np.sum(pers)}

stats_h0 = pers_stats(h0)
stats_h1 = pers_stats(h1)
print('H0 stats:', stats_h0)
print('H1 stats:', stats_h1)

In [ ]:
# for ML we want per-tract features. compute local per-tract diagrams using neighborhoods (knn) then compute stats per tract.
# 13) compute per-tract local diagrams using knn neighborhoods on feature space (PCA of socioeconomic features + coords)
print('computing local per-tract diagrams via knn neighborhoods...')
# Build per-tract feature vectors for neighbor search: use centroids + standardized socioeconomic features
feat_arr = np.vstack([coords_ordered[:,0], coords_ordered[:,1], merged['poverty_s'].to_numpy(), merged['income_s'].to_numpy(), merged['density_s'].to_numpy()]).T

# optional dim reduction
pca = PCA(n_components=min(8, feat_arr.shape[1]))
feat_pca = pca.fit_transform(feat_arr)

from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=min(K_NEIGHBORS, feat_pca.shape[0]-1)).fit(feat_pca)
neighbors = nn.kneighbors(return_distance=False)

local_feature_list = []
for i in range(feat_pca.shape[0]):
    idxs = neighbors[i]
    local_pts = feat_pca[idxs]
    # compute local weighted distance using same formula but on the subset
    # compute geo distances on subset centroids
    sub_geo = D_geo[np.ix_(idxs, idxs)]
    sub_pov = squareform(pdist(merged['poverty_s'].to_numpy()[idxs].reshape(-1,1), metric='cityblock'))
    sub_inc = squareform(pdist(merged['income_s'].to_numpy()[idxs].reshape(-1,1), metric='cityblock'))
    sub_den = squareform(pdist(merged['density_s'].to_numpy()[idxs].reshape(-1,1), metric='cityblock'))
    D_sub = sub_geo + alpha_poverty*sub_pov + alpha_income*sub_inc + alpha_density*sub_den
    
    # build vr complexes
    rips_sub = gd.RipsComplex(distance_matrix=D_sub, max_edge_length=np.percentile(D_sub[np.triu_indices_from(D_sub, k=1)],90))
    st_sub = rips_sub.create_simplex_tree(max_dimension=1)
    pers_sub = st_sub.persistence()
    
    # collect H1 stats
    h1_sub = [d[1] for d in pers_sub if d[0]==1]
    if len(h1_sub)==0:
        local_stats = [0,0,0]
    else:
        arr = np.array(h1_sub)
        pers_vals = arr[:,1] - arr[:,0]
        local_stats = [len(pers_vals), float(np.mean(pers_vals)), float(np.max(pers_vals))]
    local_feature_list.append(local_stats)

local_feat_df = pd.DataFrame(local_feature_list, columns=['h1_n','h1_mean','h1_max'])
local_feat_df['GEOID'] = merged['GEOID'].values
local_feat_df.to_csv(os.path.join(OUTPUT_DIR,'gudhi_local_persistence_features.csv'), index=False)
print('Saved per-tract local features; shape:', local_feat_df.shape)

In [ ]:
# 14) build ML dataset and run sample classifier
print('Building ML dataset using per-tract persistence stats...')
ml_df = merged.reset_index(drop=True).merge(local_feat_df, on='GEOID', how='left')
ml_df[['h1_n','h1_mean','h1_max']] = ml_df[['h1_n','h1_mean','h1_max']].fillna(0)

# create target: low median income (below state median)
if 'median_income' in ml_df.columns:
    ml_df['target_low_income'] = (ml_df['median_income'] < ml_df['median_income'].median()).astype(int)
    y = ml_df['target_low_income'].values
else:
    y = None

feature_cols = ['h1_n','h1_mean','h1_max','poverty_s','income_s','density_s']
X = ml_df[feature_cols].astype(float).fillna(0).values

if y is not None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    cv = cross_val_score(clf, X, y, cv=5).mean()
    print('RandomForest test accuracy:', acc)
    print('5-fold CV accuracy:', cv)
    
    # feature importances
    fi = pd.Series(clf.feature_importances_, index=feature_cols).sort_values(ascending=False)
    print('Feature importances:')
    print(fi)
else:
    print('No target column detected; provide median_income or another target for supervised tasks.')

# saving ML dataset
ml_df.to_csv(os.path.join(OUTPUT_DIR,'ml_dataset_gudhi.csv'), index = False)

In [ ]:
# 15) mapping: choropleth of H1 mean persistence and poverty
print('Mapping: creating choropleth (H1 mean persistence and poverty)')
# ensure geometry in gdf (loaded earlier)
if 'gdf' not in locals():
    if os.path.exists(GEOJSON_PATH):
        gdf = gpd.read_file(GEOJSON_PATH)
    else:
        gdf = gpd.read_file(TIGER_SHP)

if 'GEOID' not in gdf.columns:
    gdf['GEOID'] = gdf['STATEFP'] + gdf['COUNTYFP'] + gdf['TRACTCE']

gdf = gdf.merge(local_feat_df, on='GEOID', how='left')

gdf = gdf.to_crs(epsg=3857)

fig, axes = plt.subplots(1,2, figsize=(16,8))
gdf.plot(column='h1_mean', ax=axes[0], legend=True, cmap='OrRd', edgecolor='0.2', linewidth=0.1)
axes[0].set_title('Per-tract H1 mean persistence (GUDHI local k-NN)')

if 'poverty_rate' in merged.columns:
    # join poverty_rate to gdf
    pov_df = merged[['GEOID','poverty_rate']]
    gdf = gdf.merge(pov_df, on='GEOID', how='left')
    gdf.plot(column='poverty_rate', ax=axes[1], legend=True, cmap='YlGnBu', edgecolor='0.2', linewidth=0.1)
    axes[1].set_title('Poverty rate')
else:
    axes[1].axis('off')

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# saving our results
print('Saving artifacts...')
np.save(os.path.join(OUTPUT_DIR,'D_weighted.npy'), D_weighted)
with open(os.path.join(OUTPUT_DIR,'gudhi_persistence_global.json'),'w') as f:
    import json
    json.dump([(int(d[0]), [float(d[1][0]), float(d[1][1]) if d[1][1]!=float('inf') else None]) for d in pers], f)

print('All artifacts saved to', OUTPUT_DIR)